<a href="https://colab.research.google.com/github/kridtapon/KO-BB-Breakout/blob/main/KO_BB_Breakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.8 MB/s eta 0:00:00


In [2]:
pip install --upgrade yfinance

In [19]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Klinger Oscillator
def calculate_klinger_oscillator(df, fast_period=34, slow_period=55):
    """
    Calculate the Klinger Oscillator (KO).
    """
    # Calculate the price changes
    price_change = df['Close'].diff()
    # Calculate the volume changes
    volume_change = df['Volume'].diff()

    # Fast and slow moving averages of volume
    fast_ema = volume_change.ewm(span=fast_period).mean()
    slow_ema = volume_change.ewm(span=slow_period).mean()

    # Klinger Oscillator (KO) calculation
    klinger_oscillator = fast_ema - slow_ema
    return klinger_oscillator

# Function to calculate Bollinger Bands
def calculate_bollinger_bands(df, period=20, std_dev=2):
    """
    Calculate Bollinger Bands.
    """
    rolling_mean = df['Close'].rolling(window=period).mean()
    rolling_std = df['Close'].rolling(window=period).std()

    upper_band = rolling_mean + (rolling_std * std_dev)
    lower_band = rolling_mean - (rolling_std * std_dev)
    return upper_band, lower_band

# Define the stock symbol and time period
symbol = 'META'  # SPY is the symbol for the S&P 500 ETF
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate Klinger Oscillator
df['Klinger_Oscillator'] = calculate_klinger_oscillator(df)

# Calculate Bollinger Bands
df['Upper_Band'], df['Lower_Band'] = calculate_bollinger_bands(df)

# Bollinger Bands Squeeze: Calculate the range between the bands and check if it’s at a 20-period low
df['BB_Squeeze'] = df['Upper_Band'] - df['Lower_Band']
df['BB_Squeeze_Low'] = df['BB_Squeeze'].rolling(window=20).min()

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Entry Condition: Bollinger Bands Squeeze + Klinger Oscillator rising (KO > 0)
df['Entry'] = (df['BB_Squeeze'] == df['BB_Squeeze_Low']) & (df['Klinger_Oscillator'] > 0)

# Exit Condition: Klinger Oscillator starts to decline or price reverses inside Bollinger Bands
df['Exit'] = (df['Klinger_Oscillator'] < 0) & (df['Close'] > df['Upper_Band'])

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           247230.252699
Total Return [%]                       147.230253
Benchmark Return [%]                   180.172876
Max Gross Exposure [%]                      100.0
Total Fees Paid                       2295.578998
Max Drawdown [%]                        73.724329
Max Drawdown Duration                       623.0
Total Trades                                    8
Total Closed Trades                             7
Total Open Trades                               1
Open Trade PnL                       31305.013982
Win Rate [%]                            71.428571
Best Trade [%]                          73.459063
Worst Trade [%]                        -48.594341
Avg Winning Trade [%]                   42.811656
Avg Losing Trade [%]                   -35.154114
